In [ ]:
%pip install tensorflow
%pip install matplotlib
%pip install Pillow

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import os, cv2

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus: tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
data_dir = 'data'
image_exts = ['.bmp', '.jpeg', '.jpg', '.png']

In [ ]:
os.listdir(os.path.join(data_dir))

In [ ]:
def data_preprocessor(data_path, extension_array, new_size):
    for img_class in os.listdir(os.path.join(data_path)):
        for img in os.listdir(os.path.join(data_path, img_class)):
            img_path = os.path.join(data_path, img_class, img)
            try:
                img = cv2.imread(img_path)
                tip = os.path.splitext(img_path)[1]
                if not tip in extension_array:
                    os.remove(img_path)
                else:
                    image = Image.open(img_path)
                    resized_image = image.resize(new_size)
                    resized_image.save(img_path)
            except:
                print(f"Issue with image {img_path}")

In [ ]:
data_preprocessor(data_dir, image_exts, (256,256))

In [ ]:
data = tf.keras.utils.image_dataset_from_directory('data')

In [ ]:
iterator = data.as_numpy_iterator()
batch = iterator.next()
fig, ax = plt.subplots(nrows=4, ncols = 5, figsize=(20, 20))
for idx, img in enumerate(batch[0][:20]):
    ax.flat[idx].imshow(img.astype(int))
    ax.flat[idx].axis('off')
    ax.flat[idx].title.set_text(batch[1][idx])

In [ ]:
data = data.map(lambda x, y: (x / 255, y))

In [ ]:
print(data.as_numpy_iterator().next()[0].min())
print(data.as_numpy_iterator().next()[0].max())

In [ ]:
iterator = data.as_numpy_iterator()
batch = iterator.next()
fig, ax = plt.subplots(nrows=4, ncols = 5, figsize=(20, 20))
for idx, img in enumerate(batch[0][:20]):
    ax.flat[idx].imshow(img)
    ax.flat[idx].axis('off')
    ax.flat[idx].title.set_text(batch[1][idx])

In [ ]:
train_size = 4
test_size = 2
val_size = 1

In [ ]:
train = data.take(train_size)
test = data.skip(train_size).take(test_size)
val = data.skip(train_size + test_size).take(val_size)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout

In [ ]:
model = Sequential([
    Conv2D(16, (3,3), 1, activation='relu', input_shape = (256, 256, 3)),
    MaxPooling2D(),
    Conv2D(32, (3,3), 1, activation='relu'),
    MaxPooling2D(),
    Conv2D(16, (3,3), 1, activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(256, activation='relu'),
    Dense(1, activation='sigmoid')
])
model.compile('adam', loss=tf.losses.BinaryCrossentropy(), metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir='logs')

In [ ]:
hist = model.fit(train, epochs=20, validation_data = val, callbacks=[tensorboard_callback])

In [ ]:
hist.history

In [ ]:
fig = plt.figure()
plt.plot(hist.history['loss'], color='teal', label='loss')
plt.plot(hist.history['val_loss'], color='orange', label='val_loss')
fig.suptitle('Loss', fontsize=20)
plt.legend(loc='upper left')
plt.show()
fig = plt.figure()
plt.plot(hist.history['accuracy'], color='teal', label='accuracy')
plt.plot(hist.history['val_accuracy'], color='orange', label='val_accuracy')
fig.suptitle('Accuracy', fontsize=20)
plt.legend(loc='upper left')
plt.show()

In [ ]:
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy

In [ ]:
pre = Precision()
re = Recall()
acc = BinaryAccuracy()

In [ ]:
for batch in test.as_numpy_iterator():
    X, y = batch
    yhat = model.predict(X)
    pre.update_state(y, yhat)
    re.update_state(y, yhat)
    acc.update_state(y, yhat)

In [ ]:
print(f"Precision:{pre.result().numpy()}")
print(f"Recall:{re.result().numpy()}")
print(f"Accuracy:{acc.result().numpy()}")

In [ ]:
def predict_image(image_path, model, image_size=(256,256)):
    from PIL import Image
    test_image = Image.open(image_path)
    resized = tf.image.resize(test_image, image_size)
    yhat = model.predict(np.expand_dims(resized/255, 0))
    if yhat > .5:
        return "Sheep"
    else:
        return "Cow"

In [ ]:
print(predict_image('cow_test.jpeg', model, (256,256)))
print(predict_image('sheep_test.jpeg', model, (256,256)))

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
model.save(os.path.join('models', 'cow_sheep_model.h5'))

In [ ]:
pretrained_model = load_model(os.path.join('models', 'cow_sheep_model.h5'))

In [ ]:
predict_image('cow_test.jpeg', pretrained_model, (256,256))